<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_FvS_QoQ_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
# change this
curr_quarter = 'Sep 2022'
prev_quarter = 'Jun 2022'
len(stock_ids)

63

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 

#stock_name = 'tcs'

    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"
    
    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')
        
        test_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")
            
        return test_list
    
    test_list = get_list(actual_link)
        
    print(len(test_list))
    n = len(test_list) 
    
    if n < 114:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        print(len(test_list))
    
    q = 0
        
    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   
        
    else:
        p = 6

    df_qreults = pd.DataFrame()
    
    if 'Gross NPA %'  in test_list:
        endkey = 'Gross NPA %'
    else:
        endkey = 'Raw PDF'

    if curr_quarter in test_list:
        quarter = curr_quarter
    else:
        quarter = prev_quarter

    #df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-p:test_list.index('Sep 2021')+1]
    #df_qreults['EPS(Q)'] = test_list[test_list.index('Raw PDF')-p-q-1:test_list.index('Raw PDF')-q]
    s = test_list.index(endkey)  - test_list.index('EPS in Rs')-2
    df_qreults['Quarter'] = test_list[test_list.index(quarter)-s:test_list.index(quarter)+1]    
    df_qreults['EPS(Q)'] = test_list[test_list.index('EPS in Rs')+1:test_list.index(endkey)]

    try:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Expenses\xa0+')-s-1:test_list.index('Expenses\xa0+')]
    except:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Interest')-s-1:test_list.index('Interest')] 
    
    
    df_qreults['Sales(Q)']  = df_qreults['Sales(Q)'].str.replace(",", "")

    df_qreults['EPS(Q)'] = df_qreults['EPS(Q)'].astype(float).fillna(0.0)
    df_qreults['Sales(Q)'] = df_qreults['Sales(Q)'].astype(float).fillna(0.0)

    from pandas.tseries.offsets import MonthEnd
    import datetime

    df_qreults['Quarter'] = pd.to_datetime(df_qreults['Quarter'], format="%b %Y") + MonthEnd(1) + datetime.timedelta(days=46)
    df_qreults

    df_qreults['EPS(TTM)'] = df_qreults['EPS(Q)'] + df_qreults['EPS(Q)'].shift(1) + df_qreults['EPS(Q)'].shift(2) + \
                                df_qreults['EPS(Q)'].shift(3)
    
    df_qreults['Sales(TTM)'] = df_qreults['Sales(Q)'] + df_qreults['Sales(Q)'].shift(1) + df_qreults['Sales(Q)'].shift(2) + \
                                df_qreults['Sales(Q)'].shift(3)

    import pandas_datareader.data as web
    import yfinance as yfin
    yfin.pdr_override()

    df_stock = pd.DataFrame()

    for date in df_qreults['Quarter']:
        print(date)

        end_date = date
        start_date = (date + datetime.timedelta(days=-7)).strftime('%Y-%m-%d')


        stock_id = stock_name.upper() + '.NS'
        tmp = pd.DataFrame(web.DataReader(tickers = stock_id, data_source='yahoo', start = start_date, end = end_date))
        tmp = tmp.tail(1)
        tmp['Stock'] = stock_id
        tmp['Quarter'] = end_date  
        tmp['Close'] = round(tmp['Close'],2)
        tmp.reset_index(inplace = True)
        cols = ['Stock','Quarter','Close']
        tmp =tmp[cols]    
        df_stock = df_stock.append(tmp)

    df_stock_analysis = df_stock.merge(df_qreults, on='Quarter').tail(6)
    df_stock_analysis['PE(TTM)'] = round(df_stock_analysis['Close']/df_stock_analysis['EPS(TTM)'],2)
    
    df_analysis = df_analysis.append(df_stock_analysis)
    
df_analysis

ASIANPAINT
172
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-0

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM)
7,ASIANPAINT.NS,2021-08-15,2988.70,5.93,5585.0,36.38,24374.0,82.15
8,ASIANPAINT.NS,2021-11-15,3122.90,6.21,7096.0,33.93,26120.0,92.04
9,ASIANPAINT.NS,2022-02-15,3143.45,10.59,8527.0,31.61,27859.0,99.44
10,ASIANPAINT.NS,2022-05-16,3064.00,8.87,7893.0,31.60,29101.0,96.96
11,ASIANPAINT.NS,2022-08-15,3427.85,10.60,8607.0,36.27,32123.0,94.51
...,...,...,...,...,...,...,...,...
8,ZEEL.NS,2021-11-15,312.70,2.81,1979.0,12.07,8449.0,25.91
9,ZEEL.NS,2022-02-15,249.35,3.11,2113.0,11.02,7833.0,22.63
10,ZEEL.NS,2022-05-16,239.25,1.89,2323.0,10.04,8190.0,23.83
11,ZEEL.NS,2022-08-15,241.80,1.11,1846.0,8.92,8261.0,27.11


In [5]:
df_analysis_final = pd.DataFrame()
for stock_id in df_analysis['Stock'].unique():
    tmp = df_analysis[df_analysis['Stock'] == stock_id]
    tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
    tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
    tmp['Prev_Sales'] = tmp['Sales(Q)'].shift(1)
    tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['FR%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS(TTM)'])*100/tmp['Prev_EPS(TTM)'],2)
    tmp['SR%'] = round(tmp['EPS(TTM)']*(tmp['PE(TTM)'] - tmp['Prev_PE'])*100/tmp['Prev_Close'],2)
    tmp['TR%'] = tmp['FR%'] + tmp['SR%']
    tmp['Sales%'] = round((tmp['Sales(Q)'] - tmp['Prev_Sales'])*100/tmp['Prev_Sales'],2)
    tmp['EPS%'] = round((tmp['EPS(Q)'] - tmp['Prev_EPS(Q)'])*100/tmp['Prev_EPS(Q)'],2)
    tmp['Return%'] = tmp['FR%'] + tmp['SR%']
    df_analysis_final = df_analysis_final.append(tmp)
    
df_analysis_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
7,ASIANPAINT.NS,2021-08-15,2988.70,5.93,5585.0,36.38,24374.0,82.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ASIANPAINT.NS,2021-11-15,3122.90,6.21,7096.0,33.93,26120.0,92.04,5.93,36.38,5585.0,82.15,2988.70,-6.73,11.23,4.50,27.05,4.72,4.50
9,ASIANPAINT.NS,2022-02-15,3143.45,10.59,8527.0,31.61,27859.0,99.44,6.21,33.93,7096.0,92.04,3122.90,-6.84,7.49,0.65,20.17,70.53,0.65
10,ASIANPAINT.NS,2022-05-16,3064.00,8.87,7893.0,31.60,29101.0,96.96,10.59,31.61,8527.0,99.44,3143.45,-0.03,-2.49,-2.52,-7.44,-16.24,-2.52
11,ASIANPAINT.NS,2022-08-15,3427.85,10.60,8607.0,36.27,32123.0,94.51,8.87,31.60,7893.0,96.96,3064.00,14.78,-2.90,11.88,9.05,19.50,11.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,ZEEL.NS,2021-11-15,312.70,2.81,1979.0,12.07,8449.0,25.91,2.23,10.24,1775.0,17.90,183.25,17.87,52.76,70.63,11.49,26.01,70.63
9,ZEEL.NS,2022-02-15,249.35,3.11,2113.0,11.02,7833.0,22.63,2.81,12.07,1979.0,25.91,312.70,-8.70,-11.56,-20.26,6.77,10.68,-20.26
10,ZEEL.NS,2022-05-16,239.25,1.89,2323.0,10.04,8190.0,23.83,3.11,11.02,2113.0,22.63,249.35,-8.89,4.83,-4.06,9.94,-39.23,-4.06
11,ZEEL.NS,2022-08-15,241.80,1.11,1846.0,8.92,8261.0,27.11,1.89,10.04,2323.0,23.83,239.25,-11.16,12.23,1.07,-20.53,-41.27,1.07


In [6]:
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
df_growth = df_analysis_final[cols]
df_growth[df_growth['Quarter'] == '2022-08-15'].sort_values(by='Sales%', ascending=False).head(20)

,Stock,Quarter,Sales%,EPS%,Return%
11,INDIGO.NS,2022-08-15,60.27,-36.72,23.99
11,BERGEPAINT.NS,2022-08-15,26.14,14.98,3.05
11,PIDILITIND.NS,2022-08-15,23.69,39.20,23.59
11,IRCTC.NS,2022-08-15,23.44,14.98,2.26
11,TATAPOWER.NS,2022-08-15,21.20,58.60,4.29
11,TITAN.NS,2022-08-15,21.13,54.01,18.16
11,NAUKRI.NS,2022-08-15,15.64,-71.60,27.99
11,DMART.NS,2022-08-15,14.25,50.68,34.23
11,SUNPHARMA.NS,2022-08-15,13.92,-190.52,3.50
11,HEROMOTOCO.NS,2022-08-15,12.69,-3.43,12.75


In [7]:
# df_analysis_final.to_csv('data/Portfolio_FvS_QoQ_Analysis.csv', index=False)

In [8]:
#df_growth = pd.read_csv('data/Portfolio_FvS_QoQ_Analysis.csv')
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
#df_growth = df_growth[cols]
df_growth = df_growth.dropna()
df_growth

,Stock,Quarter,Sales%,EPS%,Return%
8,ASIANPAINT.NS,2021-11-15,27.05,4.72,4.50
9,ASIANPAINT.NS,2022-02-15,20.17,70.53,0.65
10,ASIANPAINT.NS,2022-05-16,-7.44,-16.24,-2.52
11,ASIANPAINT.NS,2022-08-15,9.05,19.50,11.88
12,ASIANPAINT.NS,2022-11-15,-1.73,-23.02,-10.93
...,...,...,...,...,...
8,ZEEL.NS,2021-11-15,11.49,26.01,70.63
9,ZEEL.NS,2022-02-15,6.77,10.68,-20.26
10,ZEEL.NS,2022-05-16,9.94,-39.23,-4.06
11,ZEEL.NS,2022-08-15,-20.53,-41.27,1.07


In [9]:
df_growth[(df_growth['Sales%'] > 5) & (df_growth['EPS%'] > 5)]['Stock'].value_counts()[0:20]

MINDTREE.NS      4
LT.NS            4
DMART.NS         4
INFY.NS          3
EICHERMOT.NS     3
TITAN.NS         3
RELIANCE.NS      3
NATIONALUM.NS    3
MARUTI.NS        3
M&M.NS           3
LTI.NS           3
JUBLFOOD.NS      3
IRCTC.NS         3
ZEEL.NS          3
BAJFINANCE.NS    3
DABUR.NS         3
BHARTIARTL.NS    3
BERGEPAINT.NS    3
ASIANPAINT.NS    2
SBILIFE.NS       2
Name: Stock, dtype: int64

In [10]:
df_tmp_final = pd.DataFrame()
for stock in df_growth['Stock'].unique():
    df_tmp = df_growth[df_growth['Stock'] == stock][cols]
    df_tmp['sales_flag'] = 0
    df_tmp['eps_flag'] = 0
    df_tmp.loc[df_tmp['Sales%'] < 0, 'sales_flag'] = 1
    df_tmp.loc[df_tmp['EPS%'] < 0, 'eps_flag'] = 1
    df_tmp['sales_weight'] =  df_tmp['sales_flag']*5 + df_tmp['sales_flag'].shift(1)*4 + df_tmp['sales_flag'].shift(2)*3+df_tmp['sales_flag'].shift(3)*2+df_tmp['sales_flag'].shift(4)*1
    df_tmp['eps_weight'] =  df_tmp['eps_flag']*5 + df_tmp['eps_flag'].shift(1)*4 + df_tmp['eps_flag'].shift(2)*3+df_tmp['eps_flag'].shift(3)*2+df_tmp['eps_flag'].shift(4)*1
    df_tmp['total_weight'] = df_tmp['sales_weight'] + df_tmp['eps_weight'] 
    df_tmp = df_tmp.tail(1)
    df_tmp_final = df_tmp_final.append(df_tmp)

In [11]:
df_tmp_final[(df_tmp_final['Sales%'] > 5) & (df_tmp_final['EPS%'] > 5)].sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
12,BAJFINANCE.NS,2022-11-15,12.33,7.11,-3.87,0,0,0.0,0.0,0.0
12,FEDERALBNK.NS,2022-11-15,11.64,15.23,22.82,0,0,1.0,0.0,1.0
12,CANBK.NS,2022-11-15,11.28,24.04,31.56,0,0,2.0,1.0,3.0
12,AXISBANK.NS,2022-11-15,6.12,28.12,11.92,0,0,0.0,4.0,4.0
12,MINDTREE.NS,2022-11-15,8.94,7.79,6.55,0,0,0.0,4.0,4.0
12,M&M.NS,2022-11-15,5.13,26.27,1.48,0,0,0.0,4.0,4.0
12,LTI.NS,2022-11-15,6.94,7.20,4.98,0,0,0.0,4.0,4.0
12,HCLTECH.NS,2022-11-15,5.21,6.28,14.89,0,0,0.0,4.0,4.0
12,ICICIBANK.NS,2022-11-15,8.19,8.19,2.35,0,0,0.0,4.0,4.0
12,DABUR.NS,2022-11-15,5.81,11.24,-4.89,0,0,3.0,3.0,6.0


In [12]:
df_tmp_final.sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
12,HINDUNILVR.NS,2022-11-15,3.56,11.94,-5.29,0,0,0.0,0.0,0.0
12,BAJFINANCE.NS,2022-11-15,12.33,7.11,-3.87,0,0,0.0,0.0,0.0
12,EICHERMOT.NS,2022-11-15,3.59,7.57,9.55,0,0,0.0,0.0,0.0
12,FEDERALBNK.NS,2022-11-15,11.64,15.23,22.82,0,0,1.0,0.0,1.0
12,CANBK.NS,2022-11-15,11.28,24.04,31.56,0,0,2.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...
12,SUZLON.NS,2022-11-15,4.13,-97.51,9.31,0,1,4.0,15.0,19.0
12,TATASTEEL.NS,2022-11-15,-5.60,-80.50,-3.51,1,1,9.0,11.0,20.0
12,DIVISLAB.NS,2022-11-15,-17.74,-29.69,-12.10,1,1,9.0,12.0,21.0
12,ICICIPRULI.NS,2022-11-15,-1503.17,27.52,-13.10,1,0,11.0,10.0,21.0


In [13]:
scrip = 'DIVISLAB.NS'
df_growth[df_growth['Stock'] == scrip][cols]

,Stock,Quarter,Sales%,EPS%,Return%
8,DIVISLAB.NS,2021-11-15,1.38,8.81,-2.12
9,DIVISLAB.NS,2022-02-15,25.40,48.82,-11.66
10,DIVISLAB.NS,2022-05-16,1.00,-0.85,0.07
11,DIVISLAB.NS,2022-08-15,-10.44,-21.54,-12.95
12,DIVISLAB.NS,2022-11-15,-17.74,-29.69,-12.10
